In [ ]:
# INSTALACIONES
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [11]:
import os
import numpy as np

# CAPTURA DE VIDEOS
import cv2


# DATAFRAMES/PARQUET
import pandas as pd


# FUNCIONALIDADES DE MEDIAPIPE
import mediapipe
#Modelo para deteccion corporal
mp_holistic = mediapipe.solutions.holistic

#Utilidades para dibujar puntos/predicciones sobre la imagen
mp_drawing = mediapipe.solutions.drawing_utils

# Funcionalidades para MediaPipe

- Import de modelo
- mediapipe_detection: Deteccion de Puntos Corporales por frame/imagen
- draw_styled_landmarks: dibuja puntos corporales sobre la imagen/frame recibido
- array_from_landmarks: retorna un arreglo concatenando puntos de [cara, pose, mano izquierda, mano derecha] para los resultados de 1 frame

In [125]:
# Ejecuta la deteccion de persona/s sobre imagen
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)                 #prediction from a frame
    image.flags.writeable = True
    
    # DESCOMENTAR SI SE QUIEREN MOSTRAR FRAMES EN PANTALLA
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [13]:
                # [color1 (B), color2 (G), color3 (R), thickness, circleRadius]
leftHandStyles =  [0, 138, 255, 2, 1] #naranja
rightHandStyles = [231, 217, 0, 2, 1] #celeste
faceStyles =      [80, 110, 10, 0, 1]
poseStyles =      [70, 100, 5, 2, 1]


# Se hace uso de las utilidades de Mediapipe para dibujar sobre la imagen los puntos/conexiones correspondientes
def draw_styled_landmarks(image, results):
    #     FACE
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        mp_drawing.DrawingSpec(color=(faceStyles[0],faceStyles[1],faceStyles[2]), thickness=faceStyles[3], circle_radius=faceStyles[4])
    )
    #     POSE/BODY
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(poseStyles[0],poseStyles[1],poseStyles[2]), thickness=poseStyles[3], circle_radius=poseStyles[4])
    )
    #     LEFT HAND
    mp_drawing.draw_landmarks(
        image,
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(leftHandStyles[0],leftHandStyles[1],leftHandStyles[2]), thickness=leftHandStyles[3], circle_radius=leftHandStyles[4])
    )
    #     RIGHT HAND
    mp_drawing.draw_landmarks(
        image,
        results.right_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(rightHandStyles[0],rightHandStyles[1],rightHandStyles[2]), thickness=rightHandStyles[3], circle_radius=rightHandStyles[4])
    )

In [61]:
""" 
    Retorna un arreglo donde se concatena los puntos corporales en el siguiente orden:
                   [puntos faciales, puntos corporales, puntos mano izquierda, puntos mano derecha]
    Dimensiones -> [     468 * 2,          33 * 2,            21 * 2,                21 * 2       ]
    cada punto esta representado por el par X e Y, coordenadas normalizadas sobre la imagen.

    La normalizacion la realiza el predictor del modelo.
"""
def array_from_landmarks(results):
    # LEFT HAND array
    # len(results.left_hand_landmarks.landmark) = 21 landmarks for each hand, with 2 coordinates each landmark
    if (results.left_hand_landmarks):
        leftHandLandmarks = np.array([[result.x, result.y] for result in results.left_hand_landmarks.landmark]).flatten()
    else:
        leftHandLandmarks = np.zeros(21*2)


    # RIGHT HAND array
    # len(results.right_hand_landmarks.landmark) -> same for right hand
    if (results.right_hand_landmarks):
        rightHandLandmarks = np.array([[result.x, result.y ] for result in results.right_hand_landmarks.landmark]).flatten()
    else:
        rightHandLandmarks = np.zeros(21*2)


    # POSE array
    # len(results.pose_landmarks.landmark) -> 33 landmarks of 2 coordinates each one (X, Y)
    if (results.pose_landmarks):
        poseLandmarks = np.array([[result.x, result.y] for result in results.pose_landmarks.landmark]).flatten()
    else:
        poseLandmarks = np.zeros(33*2)


    # FACE array
    # len(results.face_landmarks.landmark) -> 468 landmarks of 2 coordinates each one
    if (results.face_landmarks):
        faceLandmarks = np.array([[result.x, result.y] for result in results.face_landmarks.landmark]).flatten()
    else:
        faceLandmarks = np.zeros(468*2)
        
    return np.concatenate((faceLandmarks, poseLandmarks, leftHandLandmarks, rightHandLandmarks))

# TRATADO DE VIDEOS

- Celda que retorna los valores maximos y minimos que tienen los videos de una seña.
- Definicion de: Path/Ruta de alamcenamiento, Lista de Palabras, Cantidad de Personas, Cantidad de Videos por Persona y numero maximo de Frames (hallado con la celda superior).
- Creacion de carpetas para almacenar los resultados de cada video
- Funcion que realiza Padding a derecha a cada video que no alcanza la cantidad de frames "maximumNumberOfFrames", esto hace que todos los arreglos generados tengan la misma longitud.
- Procesado de videos, se procesa cada frame de cada video con MediaPipe y se almacena el resultado de cada video como un arreglo de frames, donde cada frame tiene la concatenacion de los puntos corporales que entrega array_from_landmarks(). en un archivo de extension ".npy"

In [123]:
STORE_PATH2 = os.path.join('LSA64_data')

# Lista de Palabras con las que se armarán las oraciones, no se encuentran las 64 palabras
signs_list = ['nacer','comida','brillante', 'mujer', 'hijo', 'hombre', 'lejos', 'aprender', 'espumadera','amargo','leche','Uruguay','pais','donde','ninguno','nombre','perfume','sordo','comprar','encontrar', 'nave espacial']

# se toma la lista original de señas ya que ESTÁN en el orden en que se encuentran almacenadas, su indice representa correctamente el nombre del video original
full_signs_list = ['opaco', 'rojo', 'verde', 'amarillo', 'brillante', 'celeste', 'colores', 'rosa', 'mujer', 'enemigo', 'hijo', 'hombre', 'lejos','cajón','nacer','aprender','llamar','espumadera','amargo','dulce','leche','agua','comida','Argentina','Uruguay','pais','donde','apellido','burla','cumpleanos','desayuno','foto','hambre','mapa','moneda','musica','nave espacial','ninguno','nombre','paciencia','perfume','sordo','trampa','arroz','asado','caramelo','chicle','fideos','yogurt','aceptar','agradecer','apagar','aparecer','aterrizar','atrapar','ayudar','bailar','bañarse','comprar','copiar','correr','darse cuenta','dar','encontrar']
numberOfPersons = 10
numberOfVideosPerPerson = 5

In [ ]:
# Celda que retorna el valor maximo y minimo de frames que posee cada seña en sus videos.

max_frames = np.zeros(64)
min_frames = [400 for i in range(64)]
frames = []
for sign in range(len(full_signs_list)):
    if (signs_list.count(full_signs_list[sign])):
        for j in range(10):
            for k in range(5):
                # ACTUALIZAR A PATH/RUTA DONDE SE ALMACENAN LOS VIDEOS DESCARGADOS
                cap = cv2.VideoCapture(f'''C:/Users/facur/Desktop/tesis_LSA/codigos_datos_tesis/LSA64/all_cut/0{str(sign+1).zfill(2)}_0{str(j+1).zfill(2)}_00{k+1}.mp4''')
                length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                print(length)
                print(cap)
                frames.append(length)
                if length > max_frames[sign]:
                    max_frames[sign] = length
                if length < min_frames[sign]:
                    min_frames[sign] = length
print(max_frames)
print(min_frames)
i = np.argmax(max_frames)
j = np.argmin(min_frames)
print(f'''Maximo numero de frames {max_frames[i]}. Seña: {i}''')
print(f'''Minimo numero de frames {min_frames[j]}. Seña: {j}''')
print(frames)
mean = np.mean(frames)
print(f'''Promedio: {round(mean)}.''')

In [17]:
# Valor obtenido de la ejecucion de la celda previa, corresponde al maximo valor de frames sobre todos los videos

maximumNumberOfFrames = 201

# valor de padding '3' ya que los valores del arreglo estan normalizados entre [0.0, 1.0]
paddingValue = 3
def paddData():
    return [3 for j in range(1086)]

In [18]:
""" 
    ARMADO DE COLUMNAS PARA DATAFRAME 
    Columnas numericas del 0 a 1085 representando la cantidad de puntos totales.
"""
columns = [str(i) for i in range(1086)]
columns.append('sign')

In [126]:
""" 
    CELDA QUE LEE CADA VIDEO Y REALIZA FRAME A FRAME LA DETECCION DE LA PERSONA EN CADA UNO
    PARA OBTENER EL ARREGLO DE PUNTOS CORPORALES A AÑADIR EN CADA FILA DEL DATAFRAME.
    Cada video contendrá 201 filas de frames, los videos que no alcancen esta cantidad se completarán
    las filas faltantes con el valor 3 ya que Mediapipe retorna valores normalizados en el intervalo
    [0, 1].
    PRECAUCION: Como observacion luego de haber analizado los resultados obtenidos por medidaPipe
    es posible que algunos valores superen el limite superior de 1 en algunas predicciones, estos
    valores que superen el limite se ajustan al valor superior 1.
"""

dataframeRows = []

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holisticModel:
    for sign in range(len(full_signs_list)):
        if (signs_list.count(full_signs_list[sign])):
            for personNumber in range(numberOfPersons):
                for videoNumber in range(numberOfVideosPerPerson):
                    framesCount = 0
                    caption = cv2.VideoCapture(f'''C:/Users/facur/Desktop/tesis_LSA/codigos_datos_tesis/LSA64/all_cut/0{str(sign+1).zfill(2)}_0{str(personNumber+1).zfill(2)}_00{videoNumber+1}.mp4''')
                    print(f'''Leo: 0{str(sign+1).zfill(2)}_0{str(personNumber+1).zfill(2)}_00{videoNumber+1}.mp4''')
                    if (caption.isOpened() == False):
                        print("Error opening video stream or file")
                        break
                    while(caption.isOpened()):
                        ret, frame = caption.read()
                        if (ret):
                            image, results = mediapipe_detection(frame, holisticModel)

                            #   DESCOMENTAR PARA VER LOS FRAMES QUE SE VAN PROCESANDO
                            # draw_styled_landmarks(image, results)
                            # cv2.imshow('Frame',image) 

                            keypoints = array_from_landmarks(results)
                            keypoints = keypoints.tolist()
                            # Añade palabra de la seña como ultima columna
                            keypoints.append(full_signs_list[sign])

                            dataframeRows.append(keypoints)
                            framesCount = framesCount + 1
                        else:
                            break
                        if cv2.waitKey(25) & 0xFF == ord('q'):
                            # Break the loop
                            break

                    if framesCount < maximumNumberOfFrames:
                        keypoints = paddData()
                        keypoints.append(full_signs_list[sign])
                        for auxRow in range(framesCount, maximumNumberOfFrames):
                            dataframeRows.append(keypoints)
# release the video capture object
caption.release()
# Closes all the frames
cv2.destroyAllWindows()

dataframe = pd.DataFrame(dataframeRows, columns=columns)
dataframe.to_parquet(os.path.join(f'./00_raw/raw_parquet'))

Leo: 005_001_001.mp4
Leo: 005_001_002.mp4
Leo: 005_001_003.mp4
Leo: 005_001_004.mp4
Leo: 005_001_005.mp4
Leo: 005_002_001.mp4
Leo: 005_002_002.mp4
Leo: 005_002_003.mp4
Leo: 005_002_004.mp4
Leo: 005_002_005.mp4
Leo: 005_003_001.mp4
Leo: 005_003_002.mp4
Leo: 005_003_003.mp4
Leo: 005_003_004.mp4
Leo: 005_003_005.mp4
Leo: 005_004_001.mp4
Leo: 005_004_002.mp4
Leo: 005_004_003.mp4
Leo: 005_004_004.mp4
Leo: 005_004_005.mp4
Leo: 005_005_001.mp4
Leo: 005_005_002.mp4
Leo: 005_005_003.mp4
Leo: 005_005_004.mp4
Leo: 005_005_005.mp4
Leo: 005_006_001.mp4
Leo: 005_006_002.mp4
Leo: 005_006_003.mp4
Leo: 005_006_004.mp4
Leo: 005_006_005.mp4
Leo: 005_007_001.mp4
Leo: 005_007_002.mp4
Leo: 005_007_003.mp4
Leo: 005_007_004.mp4
Leo: 005_007_005.mp4
Leo: 005_008_001.mp4
Leo: 005_008_002.mp4
Leo: 005_008_003.mp4
Leo: 005_008_004.mp4
Leo: 005_008_005.mp4
Leo: 005_009_001.mp4
Leo: 005_009_002.mp4
Leo: 005_009_003.mp4
Leo: 005_009_004.mp4
Leo: 005_009_005.mp4
Leo: 005_010_001.mp4
Leo: 005_010_002.mp4
Leo: 005_010_

In [129]:
print(dataframe.axes)
dataframe.head()


[RangeIndex(start=0, stop=211050, step=1), Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '1077', '1078', '1079', '1080', '1081', '1082', '1083', '1084', '1085',
       'sign'],
      dtype='object', length=1087)]


,0,1,2,3,4,5,6,7,8,9,...,1077,1078,1079,1080,1081,1082,1083,1084,1085,sign
0,0.527323,0.335930,0.528987,0.313159,0.528064,0.318768,0.525996,0.284972,0.529301,0.305223,...,0.343579,0.367422,0.448449,0.362574,0.412896,0.361066,0.391874,0.361069,0.374148,brillante
1,0.527683,0.334210,0.528772,0.313257,0.528172,0.318417,0.525956,0.284595,0.529039,0.305241,...,0.337872,0.354952,0.443151,0.350387,0.409113,0.348080,0.387643,0.346456,0.369334,brillante
2,0.529157,0.334962,0.530400,0.313430,0.529623,0.318719,0.527275,0.284908,0.530638,0.305443,...,0.332177,0.339381,0.433850,0.334224,0.402868,0.331521,0.382506,0.329856,0.364011,brillante
3,0.529428,0.334841,0.530522,0.313358,0.529855,0.318694,0.527543,0.284725,0.530767,0.305324,...,0.324717,0.320607,0.423679,0.315196,0.392822,0.312830,0.374137,0.311790,0.357491,brillante
4,0.530169,0.335149,0.531326,0.313868,0.530596,0.319066,0.528232,0.285064,0.531562,0.305843,...,0.322127,0.306125,0.425444,0.299533,0.393563,0.295894,0.372711,0.293554,0.353886,brillante
